In [1]:
import os
from os import path
import pandas as pd
import sys
basepath = path.join(os.getcwd(), "export_us")

In [9]:
import os
import numpy as np
import pandas as pd

start, end = '2020-06-30', '2024-06-27'
stock_codes = pd.read_csv('dji_companies.csv')['Symbol']

dfs = []
for stock_code in stock_codes:
    fname = f"{stock_code}.csv"
    fpath = os.path.join(basepath, fname)
    if os.path.exists(fpath):
        df = pd.read_csv(fpath, parse_dates=['日期'])
        df.rename(columns={'日期': 'date', '开盘': 'open', '收盘': 'close', '最高': 'high', '最低': 'low', '成交量': 'volume', '成交额': 'amount'}, inplace=True)
        df['stock_code'] = stock_code
        dfs.append(df[(df['date'] >= start) & (df['date'] <= end)])

all_df = pd.concat(dfs, ignore_index=True)
all_df.set_index(['stock_code', 'date'], inplace=True)
all_df.sort_index(inplace=True)

In [3]:
all_df

open       high        low      close    volume  \
stock_code date                                                               
US000101   2020-06-30  38.500000  39.130001  38.119999  38.980000  22169400   
           2020-07-01  39.610001  40.209999  39.400002  39.700001  18269800   
           2020-07-02  40.040001  40.279999  39.169998  39.250000  23372800   
           2020-07-06  39.799999  40.369999  39.650002  40.330002  17551400   
           2020-07-07  40.040001  40.349998  40.000000  40.070000  19404700   
...                          ...        ...        ...        ...       ...   
US000130   2023-11-24  46.220001  46.340000  45.790001  46.060001    650600   
           2023-11-27  45.880001  46.290001  45.529999  45.980000   1459100   
           2023-11-28  45.840000  45.900002  45.070000  45.230000   2919900   
           2023-11-29  45.869999  46.495998  44.904999  45.029999   2848900   
           2023-11-30  45.209999  45.250000  44.250000  44.720001   3895000   

                           code  
stock_code date                  
US000101   2020-06-30  US000101  
           2020-07-01  US000101  
           2020-07-02  US000101  
           2020-07-06  US000101  
           2020-07-07  US000101  
...                         ...  
US000130   2023-11-24  US000130  
           2023-11-27  US000130  
           2023-11-28  US000130  
           2023-11-29  US000130  
           2023-11-30  US000130  

[25860 rows x 6 columns]

In [10]:
all_df

open   close    high     low    volume        amount  \
stock_code date                                                                 
AAPL       2020-06-30  85.385  86.565  86.860  85.365  35055821  1.275552e+10   
           2020-07-01  86.645  86.393  87.205  86.343  27684309  1.012571e+10   
           2020-07-02  87.328  86.393  87.983  86.275  28510367  1.046759e+10   
           2020-07-06  87.865  88.828  89.310  87.833  29663913  1.107689e+10   
           2020-07-07  89.218  88.538  90.020  88.423  28106114  1.056394e+10   
...                       ...     ...     ...     ...       ...           ...   
NVDA       2024-02-08  70.020  69.587  70.740  69.401  41442211  2.905821e+10   
           2024-02-09  70.479  72.079  72.131  70.158  43663689  3.121476e+10   
           2024-02-12  72.546  72.194  74.557  71.196  61371018  4.473393e+10   
           2024-02-13  70.346  72.074  73.396  69.566  60258015  4.338957e+10   
           2024-02-14  73.148  73.846  74.182  71.884  50491742  3.697843e+10   

                         振幅   涨跌幅    涨跌额   换手率  
stock_code date                                 
AAPL       2020-06-30  1.74  0.88  0.755  0.82  
           2020-07-01  1.00 -0.20 -0.172  0.65  
           2020-07-02  1.98  0.00  0.000  0.67  
           2020-07-06  1.71  2.82  2.435  0.69  
           2020-07-07  1.80 -0.33 -0.290  0.66  
...                     ...   ...    ...   ...  
NVDA       2024-02-08  1.91 -0.65 -0.458  1.68  
           2024-02-09  2.84  3.58  2.492  1.77  
           2024-02-12  4.66  0.16  0.115  2.49  
           2024-02-13  5.31 -0.17 -0.120  2.45  
           2024-02-14  3.19  2.46  1.772  2.05  

[6391 rows x 10 columns]

In [11]:
all_df.to_csv("all_stock_data.csv")

In [12]:
all_df_loaded = pd.read_csv("all_stock_data.csv", parse_dates=['date'])
all_df_loaded.set_index(['stock_code', 'date'], inplace=True)
all_df_loaded.sort_index(inplace=True)

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from scipy.stats import pearsonr


# 读取合并后的数据
all_df_loaded = pd.read_csv("all_stock_data.csv", parse_dates=['date'])
all_df_loaded.set_index(['stock_code', 'date'], inplace=True)
all_df_loaded.sort_index(inplace=True)


# 定义你的因子函数，输入为单个股票的DataFrame
def factor_func(df, lookback_period_vwma=20, lookback_period_momentum=10, market_regime='Neutral'):
    price_diff_sma = (df['high'] - df['low']).rolling(window=20).mean()
    volume_ema = df['volume'].ewm(span=10, adjust=False).mean()
    heuristics_matrix = price_diff_sma * volume_ema
    return heuristics_matrix

# 计算每只股票的因子值
all_df_loaded['factor'] = all_df_loaded.groupby('stock_code').apply(lambda g: factor_func(g)).reset_index(level=0, drop=True)

# 计算未来6日收益率
all_df_loaded['future_return_6d'] = all_df_loaded.groupby('stock_code')['close'].shift(-6) / all_df_loaded['close'] - 1

# 取所有日期
start_date = pd.Timestamp('2021-01-01')
end_date = pd.Timestamp('2024-01-01')
all_dates = all_df_loaded.index.get_level_values('date').unique()
all_dates = all_dates[(all_dates >= start_date) & (all_dates <= end_date)]
ic_values = []

for date in all_dates:
    daily = all_df_loaded.xs(date, level='date')
    factors = daily['factor']
    returns = daily['future_return_6d']
    mask = factors.notna() & returns.notna()
    if mask.sum() >= 10:
        ic, _ = pearsonr(factors[mask], returns[mask])
        if not np.isnan(ic):
            ic_values.append(ic)

mean_ic = np.mean(ic_values)
print(f"Mean IC: {mean_ic:.10f}")

Mean IC: -0.0259912730
